In [1]:
# Import Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [2]:
# Function to remove formatting from gatherer
def extract_name(card, card_long):
    length = len(card_long)
    good_string = card_long[length-len(card):-1]
    good_string += card_long[length-1]
    return(good_string) 

In [3]:
# Create browser to navigate oracle
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://gatherer.wizards.com/Pages/Default.aspx'
browser.visit(url)

In [4]:
# Here will be the list with all cards which will be converted from a csv.
# Temporarily, these cards are chosen so each mana symbol is represented at least once.
cards = ["Arcum's Astrolabe", "Jund Hackblade", "Swab Goblin", "Rakdos Carnarium", "Kaervek's Torch",
         "Beseech the Queen", "Thought-Knot Seer", "Apostle's Blessing", "Counterspell", "Gideon Blackblade"]

In [5]:
# This is where the analysis will be run
copies = 1


# Empty list for list of mismatched links is crafted
bad_link = []

# Empty lists are initialized
card_list = []
copies_list = []
gen_list = []
snow_list = []
c_list = []
v_list = []
two_b_list = []
phy_list = []
w_list = []
u_list = []
b_list = []
r_list = []
g_list = []
cmc_list = []
type_line_list = []
text_list = []
power_list = []
toughness_list = []
loyalty_list = []

# For loop loops through list of cards
for card in cards:
    
    # Initialize all mana values which may be duplicated at 0
    gen = 0
    snow = 0
    c = 0
    v = 0
    two_b = 0
    phy = 0
    w = 0
    u = 0
    b = 0
    r = 0
    g = 0
    
    # Navigates browser to card page.  Note that this picks the first option in the dropdown when a card name is entered.
    # It is possible that the wrong card will be grabbed, so this is verified below.
    browser.fill("ctl00$ctl00$MainContent$Content$SearchControls$CardSearchBoxParent$CardSearchBox", card)
    time.sleep(1)
    browser.links.find_by_partial_text(card).click()
    
    # Soup object is crafted at card page so html can be read
    html = browser.html
    hot_soup = BeautifulSoup(html, "html.parser")
    
    # Check to see if card navigation was successful.  Name must be extracted from formatting to check strings are equal.
    # If strings are not equal, the entry is entered into a list.  Otherwise, the program continues.
    card_long = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_nameRow").text.split("\n")[1]
    card_name = extract_name(card, card_long)
    if card != card_name:
        bad_link.append([card,card_name])
    
    # Card name is correct,so data is scraped
    else:
        
        # Check for a mana cost, if one does not exist, skip ahead
        try:
            mana_row = hot_soup.find("div", class_="row manaRow").find_all("img")
            
            # Loop through all mana symbols in the mana row, in each case the alt text will be used.
            # Mana is saved as [gen]eric, [snow], [c]olorless, [v]ariable, (two_b)rid, (phy)rexian, and
            #    [w]hite, bl[u]e, [b]lack, [r]ed, and [g]reen (hybrid, phyrexian, and 2brid mana are counted twice).
            for cost in mana_row:
                
                # Attempt to turn symbol into a number, only possible if it is generic
                try:
                    gen = int(cost["alt"])
                
                # The symbol is not generic mana
                except:
                    
                    # These three qualities are unique, and thus mutually exclusive
                    if "Snow" == cost["alt"]:
                        snow += 1
                    elif "Colorless" == cost["alt"]:
                        c += 1
                    elif "Variable Colorless" == cost["alt"]:
                        v += 1
                    
                    # The following mana types may appear in combination in the same symbol, thus are checked if the string is
                    #    in the alt text.  The intent is that mana which may be paid for in multiple ways is counted as such.
                    else:
                        if "Two or " in cost["alt"]:
                            two_b += 1
                        if "Phyrexian" in cost["alt"]:
                            phy += 1
                        if "White" in cost["alt"]:
                            w += 1
                        if "Blue" in cost["alt"]:
                            u += 1
                        if "Black" in cost["alt"]:
                            b += 1
                        if "Red" in cost["alt"]:
                            r += 1
                        if "Green" in cost["alt"]:
                            g += 1    

        
        # If no mana cost, return all mana entries as None ("None" is different from "0", see Mountain and Ornithopter)
        except:
            gen = None
            snow = None
            c = None
            v = None
            two_b = None
            phy = None
            w = None
            u = None
            b = None
            r = None
            g = None

        
        
        # Scrape a converted mana cost. If one does not exist, skip ahead
        # In order to ensure that the cmc is a number and usable, it must be altered from that on oracle.
        # Here, we abuse the fact that the highest cmc in pauper is 9, so all numbers will be 1 character.
        try:
            ugly_number = hot_soup.find_all("div", class_="row")[2].find("div", class_="value").text
            cmc = int(ugly_number[len(ugly_number) - 1])
        except:
            cmc = None
        
        # Scrape the typeline
        type_line = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_typeRow").text.split("\n")[1]
        
        # Check for a text box, if one does not exist, skip ahead
        try:
            text = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_textRow").\
                text.replace("Card Text:\n", "")
        except:
            text = None
        
        # Check for power and toughness.  If both are valid, set loyalty to zero.
        try:
            pt = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ptRow").text.split("\n")[1].split(" / ")
            power = int(pt[0])
            toughness = int(pt[1])
            loyalty = None
        # Something failed so card has no card has no pt. Next, check if card is a planeswalker.
        except:
            power = None
            toughness = None
            try:
                loyalty = int(browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ptRow").text.split("\n")[1])
            except:
                loyalty = None
    
    # Appends the card data to the relevant lists
    card_list.append(card)
    copies_list.append(copies)
    gen_list.append(gen)
    snow_list.append(snow)
    c_list.append(c)
    v_list.append(v)
    two_b_list.append(two_b)
    phy_list.append(phy)
    w_list.append(w)
    u_list.append(u)
    b_list.append(b)
    r_list.append(r)
    g_list.append(g)
    cmc_list.append(cmc)
    type_line_list.append(type_line)
    text_list.append(text)
    power_list.append(power)
    toughness_list.append(toughness)
    loyalty_list.append(loyalty)
    
    # Navigates back to search page
    browser.back()

In [6]:
card_df= pd.DataFrame({"Name": card_list,
                       "Copies": copies_list,
                       "Cost Generic": gen_list,
                       "Cost Snow": snow_list,
                       "Cost Colorless": c_list,
                       "Cost Variable": v_list,
                       "Cost 2-brid": two_b_list,
                       "Cost Phyrexian": phy_list,
                       "Cost White": w_list,
                       "Cost Blue": u_list,
                       "Cost Black": b_list,
                       "Cost Red": r_list,
                       "Cost Green": g_list,
                       "Converted Mana Cost": cmc_list,
                       "Type Line": type_line_list,
                       "Text Box": text_list,
                       "Power": power_list,
                       "Toughness": toughness_list,
                       "Loyalty": loyalty_list
                      })
card_df

,Name,Copies,Cost Generic,Cost Snow,Cost Colorless,Cost Variable,Cost 2-brid,Cost Phyrexian,Cost White,Cost Blue,Cost Black,Cost Red,Cost Green,Converted Mana Cost,Type Line,Text Box,Power,Toughness,Loyalty
0,Arcum's Astrolabe,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Snow Artifact,( can be paid with one mana from a snow perman...,NaN,NaN,NaN
1,Jund Hackblade,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,Creature — Goblin Berserker,As long as you control another multicolored pe...,2.0,1.0,NaN
2,Swab Goblin,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,Creature — Goblin Pirate,None,2.0,2.0,NaN
3,Rakdos Carnarium,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Land,Rakdos Carnarium enters the battlefield tapped...,NaN,NaN,NaN
4,Kaervek's Torch,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Sorcery,"As long as Kaervek's Torch is on the stack, sp...",NaN,NaN,NaN
5,Beseech the Queen,1,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,6.0,Sorcery,( can be paid with any two mana or with . This...,NaN,NaN,NaN
6,Thought-Knot Seer,1,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,Creature — Eldrazi,( represents colorless mana.)\nWhen Thought-Kn...,4.0,4.0,NaN
7,Apostle's Blessing,1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,Instant,( can be paid with either or 2 life.)\nTarget ...,NaN,NaN,NaN
8,Counterspell,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,Instant,Counter target spell.,NaN,NaN,NaN
9,Gideon Blackblade,1,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,Legendary Planeswalker — Gideon,"As long as it's your turn, Gideon Blackblade i...",NaN,NaN,4.0
